In [1]:
from create_dict_vocab import *
from tokenizer_extractor import *
from datacollator import *
from mvModel import *
from mvTrainer import *
from result import *
import argparse
import os
from datasets import load_dataset, load_metric
from transformers import AutoConfig, AutoModel, Wav2Vec2Config, Wav2Vec2Model

In [2]:
create_vocab = VocabDict(root_dir ='/alt-asr/yelkheir/wav2vec_arabic/test_data', out_dir ='/alt-asr/yelkheir/wav2vec_arabic/test_data', label='ref_anno')
create_vocab.process()

{'ɛ': 0,
 'ŋ': 1,
 'v': 2,
 'l': 3,
 'iː': 4,
 'g': 5,
 'ʌ': 6,
 'b': 7,
 'f': 8,
 'ʒ': 9,
 'θ': 10,
 'uː': 11,
 'z': 12,
 'eɪ': 13,
 'n': 14,
 'p': 15,
 'm': 16,
 'ᵻ': 17,
 'ɜː': 18,
 'h': 19,
 'ð': 20,
 'd': 21,
 'ɐ': 22,
 't': 23,
 'k': 24,
 's': 25,
 'ɪ': 26,
 'ɹ': 27,
 'aɪ': 28,
 'w': 29,
 '|': 30,
 '[UNK]': 31,
 '[PAD]': 32}

In [3]:
vocab_path = '/alt-asr/yelkheir/wav2vec_arabic/test_data'+'/vocab.json'
model_name = 'facebook/wav2vec2-base'

In [6]:
tokenizer_extractor = TokenizerExtractor(vocab_path)

In [4]:
data_files = {
    "train": '/alt-asr/yelkheir/wav2vec_arabic/test_data'+"/train.csv",
    "eval": '/alt-asr/yelkheir/wav2vec_arabic/test_data'+"/eval.csv",
    "test": '/alt-asr/yelkheir/wav2vec_arabic/test_data'+"/test.csv"}

dataset = load_dataset('csv',data_files=data_files, delimiter=",",cache_dir="")
train_dataset = dataset["train"]
eval_dataset = dataset["eval"]
test_dataset = dataset["test"]

Found cached dataset csv (/export/home/yelkheir/.cache/huggingface/datasets/csv/default-aea5198dbececd98/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
import librosa

def speech_file_to_array_fn(path):
    speech_array, sampling_rate = librosa.load(path, sr=16_000)
    return speech_array

def prepare_dataset(batch):
    audio = batch["path"]
    array = speech_file_to_array_fn(audio)
    batch["input_values"] = tokenizer_extractor.processor(array, sampling_rate=16000).input_values[0]
    with tokenizer_extractor.processor.as_target_processor():
        batch["labels_p"] = tokenizer_extractor.processor(batch['ref_anno']).input_ids
        batch["labels_p_ref"] = tokenizer_extractor.processor(batch['ref_ref']).input_ids
    return batch

train_dataset = train_dataset.map(prepare_dataset, num_proc = 128)
eval_dataset = eval_dataset.map(prepare_dataset, num_proc = 128)
test_dataset = test_dataset.map(prepare_dataset, num_proc = 128)

num_proc must be <= 4. Reducing num_proc to 4 for dataset of size 4.


Map (num_proc=4):   0%|          | 0/4 [00:00<?, ? examples/s]

/home/local/QCRI/yelkheir/anaconda3/envs/ssl/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/local/QCRI/yelkheir/anaconda3/envs/ssl/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


['ð', '|', 'ɛ', '|', 'ɛ', '|', 't', '|', 'd', '|', 'ɪ', '|', 'n', '|', 'ᵻ', '|', 'ɹ', '|', 'ɐ', '|', 'ð', '|', 'ᵻ', '|', 'f', '|', 'ɪ', '|', 'f', '|', 'θ', '|', 'ɐ', '|', 'n', '|', 'ɹ', '|', 'ɛ', '|', 's', '|', 't', '|', 'ᵻ', '|', 'd', '|', 'f', '|', 'ʌ', '|', 'ɹ', '|', 't', '|', 'uː', '|', 'ʌ', '|', 'ɜː', '|', 's']['ɛ', '|', 'v', '|', 'ɹ', '|', 'ɪ', '|', 'θ', '|', 'ɪ', '|', 'ŋ', '|', 'w', '|', 'ʌ', '|', 'z', '|', 'w', '|', 'ɜː', '|', 'k', '|', 'ɪ', '|', 'ŋ', '|', 'g', '|', 's', '|', 'm', '|', 'uː', '|', 't', '|', 'l', '|', 'iː', '|', 'b', '|', 'ɛ', '|', 't', '|', 'ɹ', '|', 'ð', '|', 'ɐ', '|', 'n', '|', 'aɪ', '|', 'h', '|', 'ɐ', '|', 'd', '|', 'ɪ', '|', 'k', '|', 's', '|', 'p', '|', 'ɛ', '|', 'k', '|', 't', '|', 'ᵻ', '|', 'd']

/home/local/QCRI/yelkheir/anaconda3/envs/ssl/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


...

['ð', '|', 'ᵻ', '|', 's', '|', 'h', '|', 'ɐ', '|', 'd', '|', 'ð', '|', 'ᵻ', '|', 'ɹ', '|', 'ʌ', '|', 'w', '|', 'ɪ', '|', 'l', '|', 'd', '|', 'ɜː', '|', 'n', '|', 'ᵻ', '|', 's', '|', 'p', '|', 'ɹ', '|', 'iː', '|', 'p', '|', 'ɛ', '|', 'ɹ', '|', 'd', '|', 'h', '|', 'ɪ', '|', 'm', '|', 'f', '|', 'ɹ', '|', 'ɜː', '|', 'ð', '|', 'ɪ', '|', 's', '|', 'd', '|', 'eɪ']['ᵻ', '|', 'b', '|', 'ɪ', '|', 'k', '|', 'k', '|', 'ɐ', '|', 'n', '|', 'v', '|', 'ᵻ', '|', 's', '|', 't', '|', 'ɛ', '|', 'n', '|', 't', '|', 'w', '|', 'ᵻ', '|', 'z', '|', 'ð', '|', 'ᵻ', '|', 'f', '|', 'ᵻ', '|', 's', '|', 'θ', '|', 'ɪ', '|', 'ŋ', '|', 't', '|', 'uː', '|', 'k', '|', 'ᵻ', '|', 'm', '|', 'w', '|', 'ɪ', '|', 'θ', '|', 'ɪ', '|', 'n', '|', 'h', '|', 'ɪ', '|', 's', '|', 'v', '|', 'ɪ', '|', 'ʒ', '|', 'ᵻ', '|', 'n']

['ᵻ', '|', 'b', '|', 'ɪ', '|', 'g', '|', 'k', '|', 'ɐ', '|', 'n', '|', 'v', '|', 'ᵻ', '|', 's', '|', 't', '|', 'ɛ', '|', 'n', '|', 't', '|', 'w', '|', 'ᵻ', '|', 'z', '|', 'ð', '|', 'ᵻ', '|', 'f', '|', 'ɜː', '|', '

/home/local/QCRI/yelkheir/anaconda3/envs/ssl/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


['ð', '|', 'ɛ', '|', 'ɛ', '|', 't', '|', 'd', '|', 'ɪ', '|', 'n', '|', 'ᵻ', '|', 'ɹ', '|', 'ɐ', '|', 'ð', '|', 'ᵻ', '|', 'f', '|', 'ɪ', '|', 'f', '|', 'θ', '|', 'ɐ', '|', 'n', '|', 'ɹ', '|', 'ɛ', '|', 's', '|', 't', '|', 'ᵻ', '|', 'd', '|', 'f', '|', 'ʌ', '|', 'ɹ', '|', 't', '|', 'uː', '|', 'ʌ', '|', 'ɜː', '|', 's']
['ð', '|', 'eɪ', '|', 'eɪ', '|', 't', '|', 'd', '|', 'ɪ', '|', 'n', '|', 'ɜː', '|', 'ɐ', '|', 't', '|', 'ð', '|', 'ᵻ', '|', 'f', '|', 'ɪ', '|', 'f', '|', 'θ', '|', 'ɐ', '|', 'n', '|', 'd', '|', 'ɹ', '|', 'ɛ', '|', 's', '|', 't', '|', 'ᵻ', '|', 'd', '|', 'f', '|', 'ʌ', '|', 'ɹ', '|', 't', '|', 'uː', '|', 'a', '|', 'ʊ', '|', 'ɜː', '|', 'z']


/home/local/QCRI/yelkheir/anaconda3/envs/ssl/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


['ɛ', '|', 'v', '|', 'ɹ', '|', 'ɪ', '|', 'θ', '|', 'ɪ', '|', 'ŋ', '|', 'w', '|', 'ʌ', '|', 'z', '|', 'w', '|', 'ɜː', '|', 'k', '|', 'ɪ', '|', 'ŋ', '|', 'g', '|', 's', '|', 'm', '|', 'uː', '|', 't', '|', 'l', '|', 'iː', '|', 'b', '|', 'ɛ', '|', 't', '|', 'ɹ', '|', 'ð', '|', 'ɐ', '|', 'n', '|', 'aɪ', '|', 'h', '|', 'ɐ', '|', 'd', '|', 'ɪ', '|', 'k', '|', 's', '|', 'p', '|', 'ɛ', '|', 'k', '|', 't', '|', 'ᵻ', '|', 'd']
['ɛ', '|', 'v', '|', 'ɹ', '|', 'iː', '|', 'θ', '|', 'ɪ', '|', 'ŋ', '|', 'w', '|', 'ʌ', '|', 'z', '|', 'w', '|', 'ɜː', '|', 'k', '|', 'ɪ', '|', 'ŋ', '|', 's', '|', 'm', '|', 'uː', '|', 'ð', '|', 'l', '|', 'iː', '|', 'b', '|', 'ɛ', '|', 't', '|', 'ɜː', '|', 'ð', '|', 'ɐ', '|', 'n', '|', 'aɪ', '|', 'h', '|', 'ɐ', '|', 'd', '|', 'ɪ', '|', 'k', '|', 's', '|', 'p', '|', 'ɛ', '|', 'k', '|', 't', '|', 'ᵻ', '|', 'd']


In [8]:
data_collator = MVDataCollatorBatchWisePadding(processor=tokenizer_extractor.processor, padding=True)

In [26]:
# import gc
# gc.collect()

3692

In [27]:
from mvModel import *
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=len(tokenizer_extractor.tokenizer),
    finetuning_task="name_exp"
)
classes = 768
setattr(config, 'model_name_audio', model_name)
setattr(config, 'vocab_size', len(tokenizer_extractor.tokenizer))
setattr(config, 'classifier_proj_size', len(tokenizer_extractor.tokenizer))
setattr(config, 'pad_token_id', tokenizer_extractor.tokenizer.pad_token_id)

model = SSLforJointClassification(config=config)
model.freeze_feature_extractor()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
training_args = TrainingArguments(
    output_dir='/alt-asr/yelkheir/wav2vec_arabic/test_data',
    group_by_length=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    gradient_checkpointing=True,
    remove_unused_columns=False,
    max_steps=2,
    save_steps=1000,  #100,
    eval_steps=1000,
    logging_steps=100,
    learning_rate=3e-5,
    warmup_steps=2000,
    save_total_limit=500,
    load_best_model_at_end=True
)

In [29]:
trainer = ModelTrainerJoint(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer_extractor.feature_extractor,
)

In [30]:
trainer.train()

UnboundLocalError: local variable 'inputs' referenced before assignment